In [ ]:
from data_loaders import ModularCharatersDataLoader
from models import DDPMNextTokenV2

In [ ]:
pipeline = DDPMNextTokenV2.DDPMNextTokenV2Pipeline()

dataloader = ModularCharatersDataLoader.get_modular_char_dataloader(dataset_name='QLeca/modular_characters_small',
                                                                            split='train',
                                                                            image_size=pipeline.train_config.image_size,
                                                                            batch_size=8,
                                                                            shuffle=True)

In [ ]:
pipeline.list_versions()

In [ ]:
pipeline.load_model_from_hub(run='run_2025-06-19_14-12-46',
                             epoch=5)
pipeline.set_num_class_embeds(len(dataloader.vocab))

In [ ]:
for batch in dataloader:
    input_images = batch['input']
    target_images = batch['target']
    labels = batch['label']
    outputs = pipeline(input_images=input_images, 
                       class_labels=labels,
                       num_inference_steps=1000)
    break    
    
    
    

In [ ]:
import torch
import torchvision
from torchvision.utils import make_grid

output_images = (outputs * 0.5 + 0.5).clamp(0, 1).cpu()
        
input_images = (input_images * 0.5 + 0.5).clamp(0, 1).cpu()

target_images = (target_images * 0.5  + 0.5).clamp(0, 1).cpu()

concat = torch.concat([input_images, output_images, target_images])
grid = make_grid(concat, nrow=input_images.shape[0])
img = torchvision.transforms.ToPILImage()(grid)

In [ ]:
display(img)